In [8]:
import librosa
import numpy as np
# this fetches a song from librosa
filename = librosa.example('nutcracker')

y, sr = librosa.load(filename)


In [9]:
# 2. Separate harmonic and percussive components
y_harmonic, y_percussive = librosa.effects.hpss(y)

# 3. Compute a chromagram from the harmonic component
chromagram = librosa.feature.chroma_cqt(y=y_harmonic, sr=sr)

# 4. Define the 12 chromatic pitch classes
keys = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

# 5. Define major and minor key profiles (Krumhansl-Schmuckler)
# These represent the "ideal" distribution of pitches for major and minor keys
major_profile = np.array([6.35, 2.23, 3.48, 2.33, 4.38, 4.09, 2.52, 5.19, 2.39, 3.66, 2.29, 2.88])
minor_profile = np.array([6.33, 2.68, 3.52, 5.38, 2.60, 3.53, 2.54, 4.75, 3.98, 2.69, 3.34, 3.17])

# 6. Aggregate chroma features across time
# We sum up the energy for each pitch class across the whole song
chroma_sum = np.sum(chromagram, axis=1)

# 7. Calculate correlation with all 24 major/minor keys
correlations = []
for i in range(12):
    # Calculate correlation with the major profile for each key
    major_corr = np.corrcoef(chroma_sum, np.roll(major_profile, i))[0, 1]
    correlations.append(('{} major'.format(keys[i]), major_corr))

    # Calculate correlation with the minor profile for each key
    minor_corr = np.corrcoef(chroma_sum, np.roll(minor_profile, i))[0, 1]
    correlations.append(('{} minor'.format(keys[i]), minor_corr))

# 8. Find the key with the highest correlation
best_key = max(correlations, key=lambda item: item[1])

print(f"Estimated key: {best_key[0]}")

Estimated key: E minor
